In [1]:
from bs4 import BeautifulSoup
import urllib.request, urllib.error, urllib.parse
import pandas as pd
from selenium import webdriver

In [2]:
bbc_url = 'https://www.bbc.com'
bbc_url_news = 'https://www.bbc.com/news'
world_url = 'https://edition.cnn.com/world'
politics_url = 'https://edition.cnn.com/politics'
health_url = 'https://edition.cnn.com/health'
entertainment_url = 'https://edition.cnn.com/entertainment'

In [3]:
def getParseUrl(link, genre):
    driver = getDriver()
    driver.get(link + '/news')
    button = driver.find_element_by_link_text(genre)
    button.click()
    webContent = driver.page_source
    data = BeautifulSoup(webContent, 'html.parser')
    driver.close()
    return data

In [4]:
def getParseUrlStatic(link):
    response = urllib.request.urlopen(link)
    webContent = response.read()
    
    data = BeautifulSoup(webContent, 'html.parser')
    return data

In [5]:
def getNewsLinks(data):
    headlines = data.find_all(class_ = "gs-c-promo-heading")
    links = []
    for headline in headlines:
        try:
            link = headline["href"]
            if link[:5] != 'https':
                link = bbc_url + link
            links.append(link)
        except:
            print("Cannot find the link")
    return links

In [6]:
def getDriver():
    driver = webdriver.Chrome(executable_path=r'C:\Users\DELL\Downloads\chromedriver_win32_5\chromedriver')
    return driver

In [7]:
def getStringFromTags(tag):
    data = tag.contents
    string = ""
    for content in data:
        try:
            string += content
        except:
            string += getStringFromTags(content)
    return string

In [8]:
def getAppropriateGenre(genre):
    if genre == 'Tech':
        genre = 'technology'
    elif genre == 'Climate' or genre == 'Science':
        genre = 'science'
    elif genre == 'World' or genre == 'Asia':
        genre = 'politics'
    else:
        genre = 'entertainment'
    return genre

In [12]:
def getNewsFromLink(link, genre):
    news = {}
    try:
        data = getParseUrlStatic(link)
        news = {}
        news['title'] = data.find('h1', {'id': 'main-heading'}).contents[0]
        news['summary'] = ""
        divs = data.findAll('div', {'data-component': 'text-block'})
        for div in divs:
            news['summary'] += getStringFromTags(div)
        news['image_link'] = data.find('img')['src']
        news['link'] = link
        news['genre'] = getAppropriateGenre(genre)
        if news['summary'] == '':
            news = {}
    except:
        print("Cannot get this news " + link)
    return news

In [13]:
def getDataFromUrl(url, genre):
    data = getParseUrl(url, genre)
    links = getNewsLinks(data)
    news = [getNewsFromLink(link, genre) for link in links]
    return list(filter(None, news))

In [14]:
news = []
for genre in ['Climate', 'World', 'Asia', 'Tech', 'Business', 'Science', 'Entertainment & Arts']:
    cur_news = getDataFromUrl(bbc_url, genre)
    for new in cur_news:
        news.append(new)

Cannot get this news https://www.bbc.co.uk/news/resources/idt-d6338d9f-8789-4bc2-b6d7-3691c0e7d138


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Cannot get this news https://www.bbc.co.uk/sustainability/
Cannot get this news https://www.bbc.com/future/article/20211029-why-mandatory-vaccination-is-nothing-new
Cannot get this news https://www.bbc.com/worklife/article/20211101-why-demisexuality-is-as-real-as-any-sexual-orientation
Cannot get this news https://www.bbc.com/travel/article/20211031-how-a-portuguese-fishing-village-tamed-a-100ft-wave
Cannot get this news https://www.bbc.com/culture/article/20211029-why-hollywood-is-shunning-sex
Cannot get this news https://www.bbc.com/future/article/20211103-can-renewables-replace-fossil-fuels
Cannot get this news https://www.bbc.com/worklife/article/20211029-the-people-penalised-for-expressing-feelings-at-work
Cannot get this news https://www.bbc.com/travel/article/20211101-varanasi-where-its-forbidden-to-serve-meat
Cannot get this news https://www.bbc.com/sport/athletics/59195113
Cannot get this news https://www.bbc.com/future/article/20211029-why-mandatory-vaccination-is-nothing-new

In [15]:
len(news)

198

In [19]:
pd.DataFrame(news).dropna()

,title,summary,image_link,link,genre
0,COP26: Time to sober up,So how much progress has really been made in t...,https://ichef.bbci.co.uk/news/976/cpsprodpb/41...,https://www.bbc.com/news/science-environment-5...,science
1,COP26: Time to sober up,So how much progress has really been made in t...,https://ichef.bbci.co.uk/news/976/cpsprodpb/41...,https://www.bbc.com/news/science-environment-5...,science
2,Climate change: National Trust joins internati...,The National Trust is calling on the governmen...,https://ichef.bbci.co.uk/news/976/cpsprodpb/63...,https://www.bbc.com/news/uk-59195535,science
3,COP26: Thousands march for Glasgow's biggest p...,"About 100,000 people marched in Glasgow to dem...",https://ichef.bbci.co.uk/news/976/cpsprodpb/BC...,https://www.bbc.com/news/uk-scotland-59185007,science
4,COP26: Why we marched in Glasgow,Tens of thousands of people have braved the we...,https://ichef.bbci.co.uk/news/976/cpsprodpb/43...,https://www.bbc.com/news/uk-scotland-59191466,science
...,...,...,...,...,...
193,"Film & TV: Atlas maps Hogwarts, Jurassic Park ...",So where actually is Batman's Gotham City? Or ...,https://ichef.bbci.co.uk/news/976/cpsprodpb/FA...,https://www.bbc.com/news/uk-wales-59151209,entertainment
194,How pop star Zara Larsson made a seven-figure ...,Pop star Zara Larsson says she's made a seven-...,https://ichef.bbci.co.uk/news/976/cpsprodpb/17...,https://www.bbc.com/news/entertainment-arts-59...,entertainment
195,Voyage: 'Vintage Abba' or 'bafflingly retrogra...,"Forty years after their last studio album, Abb...",https://ichef.bbci.co.uk/news/976/cpsprodpb/12...,https://www.bbc.com/news/entertainment-arts-59...,entertainment
196,Give Us A Clue to Extras: The life of Lionel B...,Lionel Blair will be remembered as a variety e...,https://ichef.bbci.co.uk/news/976/cpsprodpb/10...,https://www.bbc.com/news/entertainment-arts-48...,entertainment
